In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
churn = pd.read_csv("../input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv")
churn.head()

# **Разведочный анализ**

In [ ]:
print(churn["customer service calls"].value_counts())
churn["customer service calls"].hist()
plt.show()

In [ ]:
print(churn["total intl calls"].value_counts())
churn["total intl calls"].hist()
plt.show()

# **Preprocessing**

In [ ]:
churn["international plan"] = churn["international plan"].map({"yes":1, "no": 0})
churn["voice mail plan"] = churn["voice mail plan"].map({"yes":1, "no":0})
churn['churn'] = churn['churn'].map({True:1, False:0})
churn.head()

In [ ]:
df = churn.drop(["state", "phone number"], axis = 1)
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler


X = df.drop("churn", axis = 1)
y = df["churn"]

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X_scaled,
                                                      y, 
                                                      test_size = 0.25, 
                                                      random_state = 19)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier(n_neighbors = 1) 
knn.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score


y_pred = knn.predict(X_valid)
# print(y_pred)

accuracy_score(y_valid, y_pred)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


kf = KFold(n_splits = 5, shuffle = True, random_state = 42)
knn = KNeighborsClassifier(n_neighbors = 5)
scores = cross_val_score(knn, X, y, cv = kf, scoring = 'accuracy')
mean_score = scores.mean()

print(scores)
print(mean_score)

In [ ]:
from sklearn.model_selection import GridSearchCV
knn_params = {'n_neighbors': np.arange(1, 51)}
knn_grid = GridSearchCV(knn, 
                        knn_params, 
                        scoring='accuracy',
                        cv=5) # или cv=kf
knn_grid.fit(X_train, y_train)

In [ ]:
print(knn_grid.best_params_)
print(knn_grid.best_score_)

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors = 5)
y_pred = best_knn.fit(X_train, y_train).predict(X_valid)
accuracy_score(y_valid, y_pred)

In [ ]:
x_values = np.arange(1, 51)
y_values = pd.DataFrame(knn_grid.cv_results_)["mean_test_score"]

plt.plot(x_values, y_values)
plt.show()